In [ ]:
# Step 1: Setup and Imports
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

In [ ]:
# Step 2: Load and Preprocess CIFAR-10 Dataset
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = DataLoader(trainset, batch_size=100, shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = DataLoader(testset, batch_size=100, shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')


100%|██████████| 170498071/170498071 [00:03<00:00, 43644258.96it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [ ]:
# Step 3: Define the CNN Model
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, 3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(64 * 8 * 8, 512)
        self.fc2 = nn.Linear(512, 10)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.pool(self.relu(self.conv1(x)))
        x = self.pool(self.relu(self.conv2(x)))
        x = x.view(-1, 64 * 8 * 8)
        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [ ]:
net = Net()

# Step 4: Specify Loss Function and Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=0.001)


In [ ]:
# Step 5: Train the Model
def train_model(net, trainloader, criterion, optimizer, epochs=10):
    for epoch in range(epochs):
        running_loss = 0.0
        for i, data in enumerate(trainloader, 0):
            inputs, labels = data
            optimizer.zero_grad()
            outputs = net(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            if i % 100 == 99:
                print(f'[Epoch {epoch + 1}, Batch {i + 1}] loss: {running_loss / 100:.3f}')
                running_loss = 0.0
    print('Finished Training')

In [ ]:
# Step 6: Evaluate the Model
def evaluate_model(net, testloader):
    correct = 0
    total = 0
    with torch.no_grad():
        for data in testloader:
            images, labels = data
            outputs = net(images)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    print(f'Accuracy of the network on the 10000 test images: {100 * correct / total:.2f}%')


In [ ]:
# Training the model
train_model(net, trainloader, criterion, optimizer, epochs=5)

# Evaluating the model
evaluate_model(net, testloader)

# Saving the trained model
torch.save(net.state_dict(), 'cifar10_cnn.pth')

[Epoch 1, Batch 100] loss: 1.690
[Epoch 1, Batch 200] loss: 1.328
[Epoch 1, Batch 300] loss: 1.167
[Epoch 1, Batch 400] loss: 1.093
[Epoch 1, Batch 500] loss: 1.011
[Epoch 2, Batch 100] loss: 0.906
[Epoch 2, Batch 200] loss: 0.877
[Epoch 2, Batch 300] loss: 0.865
[Epoch 2, Batch 400] loss: 0.851
[Epoch 2, Batch 500] loss: 0.830
[Epoch 3, Batch 100] loss: 0.680
[Epoch 3, Batch 200] loss: 0.696
[Epoch 3, Batch 300] loss: 0.674
[Epoch 3, Batch 400] loss: 0.678
[Epoch 3, Batch 500] loss: 0.667
[Epoch 4, Batch 100] loss: 0.485
[Epoch 4, Batch 200] loss: 0.515
[Epoch 4, Batch 300] loss: 0.517
[Epoch 4, Batch 400] loss: 0.510
[Epoch 4, Batch 500] loss: 0.507
[Epoch 5, Batch 100] loss: 0.326
[Epoch 5, Batch 200] loss: 0.326
[Epoch 5, Batch 300] loss: 0.357
[Epoch 5, Batch 400] loss: 0.377
[Epoch 5, Batch 500] loss: 0.376
Finished Training
Accuracy of the network on the 10000 test images: 73.74%
